In [1]:
import sys
!{sys.executable} -m pip install transformers pillow torch lmdb pydantic

In [3]:
import os
import sys
from importlib import reload
module_path = os.path.abspath(os.path.join('/workspaces/vector_db/jfdb'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [15]:
import jfdb
reload(jfdb)

<module 'jfdb' from '/workspaces/vector_db/jfdb/__init__.py'>

In [16]:
from jfdb import hsnw

PydanticSchemaGenerationError: Unable to generate pydantic-core schema for <class 'jfdb.nodes.node.Node'>. Set `arbitrary_types_allowed=True` in the model_config to ignore this error or implement `__get_pydantic_core_schema__` on your type to fully support it.

If you got this error by calling handler(<some type>) within `__get_pydantic_core_schema__` then you likely need to call `handler.generate_schema(<some type>)` since we do not call `__get_pydantic_core_schema__` on `<some type>` otherwise to avoid infinite recursion.

For further information visit https://errors.pydantic.dev/2.9/u/schema-for-unknown-type

In [17]:
from jfdb.nodes.node import Node

In [18]:
Node

jfdb.nodes.node.Node

In [19]:
from jfdb.backend.backend import Backend

In [1]:
from jfdb.hsnw import DataBase

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
